In [ ]:
# count=0
# for i in range(10):
#     for j in range(10):
#         count+=1
#         # Extract the average red values for the current block
#         signal = selected_frames[i, j, :]
#         print(f'Block {count} - Signal Shape: {signal.shape}')
#         # print(signal.shape)
#         print(signal)
        

In [ ]:
# from scipy.signal import butter, filtfilt
# # Define Butterworth filter parameters
# nyquist_freq = fps / 2  # Nyquist frequency
# cutoff_freq = 0.5  # Desired cutoff frequency in Hz
# order = 2  # Filter order

# # Compute normalized cutoff frequency
# normalized_cutoff_freq = cutoff_freq / nyquist_freq

# # Apply Butterworth filter
# b, a = butter(order, normalized_cutoff_freq, btype='low', analog=False)
# filtered_signals = filtfilt(b, a, Su, axis=2)

In [ ]:
# import numpy as np
# from scipy.fft import fft, ifft

# # Define the frequency range
# low_freq = 0.5  # Lower cutoff frequency in Hz
# high_freq = 5.0  # Higher cutoff frequency in Hz

# # Compute the frequency corresponding to each index of the FFT
# n = len(Su[0, 0])  # Length of each PPG signal
# freq = np.fft.fftfreq(n)

# # Initialize the filtered PPG signals
# Sc = np.zeros_like(Su)

# # Apply FFT to each PPG signal
# for i in range(Su.shape[0]):  # Iterate over blocks
#     for j in range(Su.shape[1]):
#         ppg_signal = Su[i, j, :]
#         fft_signal = fft(ppg_signal)
        
#         # Identify indices corresponding to frequencies outside the desired range
#         unwanted_indices = (freq < low_freq) | (freq > high_freq)
        
#         # Set those frequency components to zero
#         fft_signal[unwanted_indices] = 0
        
#         # Apply inverse FFT to get the filtered signal
#         filtered_signal = ifft(fft_signal)
        
#         # Store the filtered signal
#         Sc[i, j, :] = filtered_signal.real  # Take the real part to avoid imaginary components

# # Sc now contains the filtered PPG signals


In [ ]:
from scipy.signal import butter, filtfilt
from scipy import fftpack

import numpy as np

# Sampling rate (r) and duration of the signal need to be defined based on your video analysis
r = fps  # frames per second from video metadata
duration = frame_count / r  # Duration of the video in seconds

# Generate a Butterworth filter
lowcut = 0.5  # Low frequency threshold (Hz)
highcut = 5.0  # High frequency threshold (Hz)
nyquist = 0.5 * r
low = lowcut / nyquist
high = highcut / nyquist
b, a = butter(3, [low, high], btype='band')

# Apply Butterworth filter (assuming 'signal' is your PPG signal extracted from the video)
filtered_signal = filtfilt(b, a, selected_frames)

# Apply FFT and remove frequencies outside the desired range
W = fftpack.fft(filtered_signal)
frequencies = fftpack.fftfreq(len(filtered_signal), 1/r)

# Zero out frequencies outside the desired range
W[(frequencies < lowcut) | (frequencies > highcut)] = 0

# Get the filtered signal back
clean_signal = fftpack.ifft(W)

# Normalize and invert the signal to match hemoglobin level relationship
Sc = np.abs(clean_signal)  # Taking absolute value to handle complex result from ifft
SPG = Sc * -1  # Inverting the signal

# SPG is now your processed PPG signal ready for further analysis
SPG

In [ ]:
# Assuming SPG is of shape (10, 10, num_frames) where num_frames is the length after selecting frames 51 to 550
# Average across all blocks
avg_signal = np.mean(SPG, axis=(0, 1))  # This averages the signal across the first two dimensions (all blocks)

plt.figure(figsize=(10, 6))
plt.plot(avg_signal, label='Average Processed PPG Signal')
plt.title('Average Processed PPG Signal')
plt.xlabel('Time (frames)')
plt.ylabel('Signal Amplitude')
plt.legend()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

def detect_ppg_cycles_for_one_signal(sppg_signal, sampling_rate):
    ppg_cycles = []

    peaks, _ = find_peaks(sppg_signal)
    valleys, _ = find_peaks(-sppg_signal)

    # Ensure peaks and valleys are not empty
    if len(peaks) == 0 or len(valleys) == 0:
        return ppg_cycles

    # Plot the PPG signal
    plt.figure()
    plt.plot(sppg_signal, label='PPG Signal')

    # Mark peaks and valleys differently
    plt.scatter(peaks, sppg_signal[peaks], color='red', marker='o', label='Peaks')
    plt.scatter(valleys, sppg_signal[valleys], color='blue', marker='x', label='Valleys')

    # Plot the legend
    plt.legend()

    plt.xlabel('Sample Index')
    plt.ylabel('PPG Signal Value')
    plt.title('Detected Peaks and Valleys')
    plt.show()

    return ppg_cycles

# Example usage
sampling_rate = fps 
sppg_signal = SPPG[0, 0, :]  # Assuming we are processing the first PPG signal
ppg_cycles = detect_ppg_cycles_for_one_signal(sppg_signal, sampling_rate)
print("Detected PPG cycles:", ppg_cycles)


In [ ]:
import matplotlib.pyplot as plt

def detect_ppg_cycles_for_one_signal(sppg_signal, sampling_rate):
    ppg_cycles = []

    peaks, _ = find_peaks(sppg_signal)
    valleys, _ = find_peaks(-sppg_signal)

    # Ensure peaks and valleys are not empty
    if len(peaks) == 0 or len(valleys) == 0:
        return ppg_cycles

    # Plot the PPG signal
    plt.figure()
    plt.plot(sppg_signal, label='PPG Signal')

    # Calculate peak and valley heights
    peak_heights = sppg_signal[peaks]
    valley_heights = sppg_signal[valleys]

    # Find the maximum peak and minimum valley heights
    max_peak_height = max(peak_heights)
    min_valley_height = min(valley_heights)

    # Mark peaks and valleys differently based on their heights
    for peak, peak_height in zip(peaks, peak_heights):
        if peak_height == max_peak_height:
            marker = 's'  # Systolic peak marker style
            color = 'red'  # Systolic peak color
        else:
            marker = 'o'  # Diastolic peak marker style
            color = 'blue'  # Diastolic peak color
        plt.scatter(peak, sppg_signal[peak], color=color, marker=marker, label='Peaks')

    for valley in valleys:
        plt.scatter(valley, sppg_signal[valley], color='green', marker='x', label='Valleys')

    # Plot the legend
    plt.legend()

    plt.xlabel('Sample Index')
    plt.ylabel('PPG Signal Value')
    plt.title('Detected Peaks and Valleys')
    plt.show()

    return ppg_cycles

# Example usage
sampling_rate = fps 
sppg_signal = SPPG[0, 0, :]  # Assuming we are processing the first PPG signal
ppg_cycles = detect_ppg_cycles_for_one_signal(sppg_signal, sampling_rate)
print("Detected PPG cycles:", ppg_cycles)


In [ ]:
import matplotlib.pyplot as plt

def detect_ppg_cycles_for_one_signal(sppg_signal, sampling_rate):
    ppg_cycles = []

    peaks, _ = find_peaks(sppg_signal)
    valleys, _ = find_peaks(-sppg_signal)

    # Ensure peaks and valleys are not empty
    if len(peaks) == 0 or len(valleys) == 0:
        return ppg_cycles

    # Find consecutive minima and maxima
    for peak_idx in range(len(peaks) - 1):
        for valley_idx in range(len(valleys) - 1):
            start_idx = valleys[valley_idx]
            end_idx = valleys[valley_idx + 1]

            # Check if the current peak is within the current valley
            if start_idx < peaks[peak_idx] < end_idx:
                systolic_peak = peaks[peak_idx]
                diastolic_peak = peaks[peak_idx + 1]
                start_point = start_idx
                end_point = valleys[valley_idx + 2]
                dicrotic_notch = valleys[valley_idx+1]

                # # Check conditions for valid PPG cycle
                # if (sppg_signal[systolic_peak] > sppg_signal[diastolic_peak] and
                #     sppg_signal[dicrotic_notch] > sppg_signal[start_point] and
                #     sppg_signal[dicrotic_notch] > sppg_signal[end_point]):

                #     # Calculate time elapsed for PPG cycle
                #     cycle_time = (end_point - start_point) / sampling_rate
                #     expected_cycle_time = get_expected_cycle_time(sppg_signal, sampling_rate)
                #     error_margin = 0.2 * expected_cycle_time

                #     # Check if time elapsed is within threshold
                #     if abs(cycle_time - expected_cycle_time) <= error_margin:
                #         ppg_cycles.append((start_point, systolic_peak, dicrotic_notch, diastolic_peak, end_point))

                # Plot the PPG signal with markers for peaks and valleys
                plt.figure()
                plt.plot(sppg_signal)
                # plt.scatter([start_point, systolic_peak, dicrotic_notch, diastolic_peak, end_point], 
                #             [sppg_signal[start_point], sppg_signal[systolic_peak], sppg_signal[dicrotic_notch], sppg_signal[diastolic_peak], sppg_signal[end_point]], 
                #             color='red', marker='o')
                

                 # Mark each point with a different marker
                plt.scatter(start_point, sppg_signal[start_point], color='green', marker='^', label='Start Point')
                plt.scatter(systolic_peak, sppg_signal[systolic_peak], color='orange', marker='s', label='Systolic Peak')
                plt.scatter(dicrotic_notch, sppg_signal[dicrotic_notch], color='purple', marker='*', label='Dicrotic Notch')
                plt.scatter(diastolic_peak, sppg_signal[diastolic_peak], color='pink', marker='d', label='Diastolic Peak')
                plt.scatter(end_point, sppg_signal[end_point], color='black', marker='P', label='End Point')
                plt.xlabel('Sample Index')
                plt.ylabel('PPG Signal Value')
                plt.title('Detected PPG Cycles')
                # plt.legend(['PPG Signal', 'Peaks and Valleys'])
                plt.legend()
                plt.show()

    return ppg_cycles

# Example usage
sampling_rate = fps 
sppg_signal = SPPG[0, 0, :]  # Assuming we are processing the first PPG signal
ppg_cycles = detect_ppg_cycles_for_one_signal(sppg_signal, sampling_rate)
print("Detected PPG cycles:", ppg_cycles)


In [ ]:
import numpy as np 
from scipy.signal import find_peaks
from scipy.fft import fft, fftfreq

def get_expected_cycle_time(ppg_signal, sampling_rate):
    # Compute the FFT of the PPG signal
    fft_signal = fft(ppg_signal)
    
    # Compute the frequency spectrum and corresponding frequencies
    n = len(ppg_signal)
    freq = fftfreq(n, d=1/sampling_rate)
    
    # Find the index of the dominant frequency component
    dominant_freq_idx = np.argmax(np.abs(fft_signal))
    
    # Calculate the period of the dominant frequency component
    dominant_freq = freq[dominant_freq_idx]
    if dominant_freq == 0:
        # Avoid division by zero when the dominant frequency is 0 Hz
        return np.inf
    expected_period = 1 / dominant_freq
    
    # Convert period to cycle time (in seconds)
    expected_cycle_time = expected_period
    
    return expected_cycle_time


def detect_ppg_cycles_for_one_signal(sppg_signal, sampling_rate):
    ppg_cycles = []

    peaks, _ = find_peaks(sppg_signal)
    valleys, _ = find_peaks(-sppg_signal)

    # Ensure peaks and valleys are not empty
    if len(peaks) == 0 or len(valleys) == 0:
        return ppg_cycles

    # Find consecutive minima and maxima
    for peak_idx in range(len(peaks) - 1):
        for valley_idx in range(len(valleys) - 1):
            start_idx = valleys[valley_idx]
            end_idx = valleys[valley_idx + 1]

            # Check if the current peak is within the current valley
            if start_idx < peaks[peak_idx] < end_idx:
                systolic_peak = peaks[peak_idx]
                diastolic_peak = peaks[peak_idx + 1]
                start_point = start_idx
                end_point = end_idx
                dicrotic_notch = valleys[valley_idx]

                # Check conditions for valid PPG cycle
                if (sppg_signal[systolic_peak] > sppg_signal[diastolic_peak] and
                    sppg_signal[dicrotic_notch] > sppg_signal[start_point] and
                    sppg_signal[dicrotic_notch] > sppg_signal[end_point]):

                    # Calculate time elapsed for PPG cycle
                    cycle_time = (end_point - start_point) / sampling_rate
                    expected_cycle_time = get_expected_cycle_time(sppg_signal, sampling_rate)
                    error_margin = 0.2 * expected_cycle_time

                    # Check if time elapsed is within threshold
                    if abs(cycle_time - expected_cycle_time) <= error_margin:
                        ppg_cycles.append((start_point, systolic_peak, dicrotic_notch, diastolic_peak, end_point))

    return ppg_cycles

# Example usage
sampling_rate = fps 
sppg_signal = SPPG[0, 0, :]  # Assuming we are processing the first PPG signal
ppg_cycles = detect_ppg_cycles_for_one_signal(sppg_signal, sampling_rate)
print("Detected PPG cycles:", ppg_cycles)


In [ ]:
import numpy as np

def detect_ppg_cycles(SPPG, sampling_rate):
    ppg_cycles = []

    for i in range(10):
        for j in range(10):
            sppg_signal = SPPG[i, j, :]
            peaks, _ = find_peaks(sppg_signal)
            valleys, _ = find_peaks(-sppg_signal)

            # Ensure peaks and valleys are not empty
            if len(peaks) == 0 or len(valleys) == 0:
                continue

            # Find consecutive minima and maxima
            for peak_idx in range(len(peaks) - 1):
                for valley_idx in range(len(valleys) - 1):
                    start_idx = valleys[valley_idx]
                    end_idx = valleys[valley_idx + 1]

                    # Check if the current peak is within the current valley
                    if start_idx < peaks[peak_idx] < end_idx:
                        systolic_peak = peaks[peak_idx]
                        diastolic_peak = peaks[peak_idx + 1]
                        start_point = start_idx
                        end_point = end_idx
                        dicrotic_notch = valleys[valley_idx]

                        # Check conditions for valid PPG cycle
                        if (sppg_signal[systolic_peak] > sppg_signal[diastolic_peak] and
                            sppg_signal[dicrotic_notch] > sppg_signal[start_point] and
                            sppg_signal[dicrotic_notch] > sppg_signal[end_point]):

                            # Calculate time elapsed for PPG cycle
                            cycle_time = (end_point - start_point) / sampling_rate
                            expected_cycle_time = get_expected_cycle_time(sppg_signal)
                            error_margin = 0.2 * expected_cycle_time

                            # Check if time elapsed is within threshold
                            if abs(cycle_time - expected_cycle_time) <= error_margin:
                                ppg_cycles.append((start_point, systolic_peak, dicrotic_notch, diastolic_peak, end_point))

    return ppg_cycles

def get_expected_cycle_time(ppg_signal):
    # Logic to calculate expected cycle time from FFT of PPG signal
    # This function needs to be implemented based on the FFT results
    return expected_cycle_time

# Example usage
sampling_rate = 30  # Sample rate in Hz (assumed)
ppg_cycles = detect_ppg_cycles(SPPG, sampling_rate)
print("Detected PPG cycles:", ppg_cycles)


In [ ]:
import numpy as np

def merge_ppg_cycles(selected_cycles):
    num_cycles = len(selected_cycles)
    # print("Number of cycles:", num_cycles)
    if num_cycles == 0:
        return None
    
    cycle_length = selected_cycles[0][4] - selected_cycles[0][0] + 1
    print("Cycle length:", cycle_length)
    print("selected_cycles[0]][0]:", selected_cycles[0][0])
    print("selected_cycles[0][4]:", selected_cycles[0][4])
    merged_signal = np.zeros(cycle_length)
    
    # Iterate over each selected cycle and add their values together
    for cycle in selected_cycles:
        start_idx, systolic_peak_idx, dicrotic_notch_idx, diastolic_peak_idx, end_idx = cycle
        cycle_signal = sppg_signal[start_idx:end_idx+1]
        print("cycle_signal:", cycle_signal)
        merged_signal[:len(cycle_signal)] += cycle_signal
    
    # Take the average of the summed signals
    merged_signal /= num_cycles
    
    return merged_signal

# Example usage
merged_signal = merge_ppg_cycles(selected_cycles)
print("Merged PPG signal:", merged_signal)


In [ ]:
# def extract_features(merged_signal, sampling_rate):
#     all_features = []

#     if merged_signal is None:
#         return None
    
#     # Smooth the merged signal using Savitzky-Golay filtering
#     smoothed_signal = savgol_filter(merged_signal, window_length=15, polyorder=3)

#     # Compute the 1st and 2nd derivatives of the smoothed signal
#     first_derivative = np.gradient(smoothed_signal)
#     second_derivative = np.gradient(first_derivative)

#     # Find peaks and valleys in the smoothed signal
#     peaks, _ = find_peaks(smoothed_signal, distance=int(sampling_rate * 0.5))
#     valleys, _ = find_peaks(-smoothed_signal, distance=int(sampling_rate * 0.5))

#     # Compute various features from the signal and its derivatives
#     features = []

#         # Systolic peak height
#     systolic_peak_height = np.max(smoothed_signal)
#     features.append(systolic_peak_height)

#     # Diatolic peak height
#     diatolic_peak_height = np.min(smoothed_signal[valleys])
#     features.append(diatolic_peak_height)

#     # Dicrotic notch height
#     dicrotic_notch_height = smoothed_signal[peaks][1] - smoothed_signal[valleys][1]
#     features.append(dicrotic_notch_height)

#     # Pulse interval
#     pulse_interval = (peaks[1] - peaks[0]) / sampling_rate
#     features.append(pulse_interval)

#     # Augmentation index
#     augmentation_index = (systolic_peak_height - diatolic_peak_height) / diatolic_peak_height * 100
#     features.append(augmentation_index)

#     # Relative augmentation index
#     relative_augmentation_index = (systolic_peak_height - diatolic_peak_height) / systolic_peak_height * 100
#     features.append(relative_augmentation_index)

#     # Ratio of z and x
#     z_x_ratio = dicrotic_notch_height / systolic_peak_height
#     features.append(z_x_ratio)

#     # Negative relative augmentation index
#     negative_relative_augmentation_index = (diatolic_peak_height - systolic_peak_height) / diatolic_peak_height * 100
#     features.append(negative_relative_augmentation_index)

#     # Systolic peak time
#     systolic_peak_time = peaks[0] / sampling_rate
#     features.append(systolic_peak_time)

#     # Dicrotic notch time
#     dicrotic_notch_time = peaks[1] / sampling_rate
#     features.append(dicrotic_notch_time)

#     # Diastolic peak time
#     diastolic_peak_time = valleys[0] / sampling_rate
#     features.append(diastolic_peak_time)

#     # Time between systolic and diastolic peaks
#     time_between_peaks = (peaks[0] - valleys[0]) / sampling_rate
#     features.append(time_between_peaks)

#     # Time between half systolic peak points
#     half_systolic_peak_time = (peaks[0] - valleys[1]) / sampling_rate
#     features.append(half_systolic_peak_time)

#     # Inflection point area ratio
#     inflection_point_area_ratio = np.trapz(smoothed_signal[valleys[0]:peaks[1]]) / np.trapz(smoothed_signal[peaks[0]:valleys[0]])
#     features.append(inflection_point_area_ratio)
    
#     # Systolic peak rising slope
#     systolic_peak_rising_slope = (smoothed_signal[peaks[0]] - smoothed_signal[valleys[0]]) / time_between_peaks
#     features.append(systolic_peak_rising_slope)
    
#     # Diatolic peak falling slope
#     diatolic_peak_falling_slope = (smoothed_signal[valleys[0]] - smoothed_signal[peaks[1]]) / time_between_peaks
#     features.append(diatolic_peak_falling_slope)
    
#     # Ratio of t1 and pulse interval time (tpi)
#     t1_tpi_ratio = time_between_peaks / pulse_interval
#     features.append(t1_tpi_ratio)

#     # Ratio of t2 and pulse interval time (tpi)
#     t2_tpi_ratio = half_systolic_peak_time / pulse_interval
#     features.append(t2_tpi_ratio)

#     # Ratio of t3 and pulse interval time (tpi)
#     t3_tpi_ratio = (peaks[1] - valleys[1]) / sampling_rate / pulse_interval
#     features.append(t3_tpi_ratio)

#     # Ratio of deltaT and pulse interval time (tpi)
#     deltaT_tpi_ratio = (valleys[1] - peaks[0]) / sampling_rate / pulse_interval
#     features.append(deltaT_tpi_ratio)
    
#     # Interval time from first PPG cycle start point (l1) in first derivative of PPF (Sf) to first maxima (a1) of Sf
#     ta1 = peaks[0] - valleys[0]
#     features.append(ta1)

#     # Interval time from point l1 to first minima of first PPG cycle (b1) in the Sf
#     tb1 = valleys[0] - peaks[0]
#     features.append(tb1)
    
#     # Interval time from point l1 to second minima of the first PPG cycle (f1) in the Sf
#     tf1 = valleys[1] - peaks[0]
#     features.append(tf1)
    
#     # Ratio of first minima (b2) and first maxima (a2) in the second derivative of PPG signal (Sf2)
#     b2_a2_ratio = second_derivative[valleys[0]] / second_derivative[peaks[0]]
#     features.append(b2_a2_ratio)
    
#     # Ratio od second maxima (e2) in Sf2 and a2
#     e2_a2_ratio = second_derivative[peaks[1]] / second_derivative[peaks[0]]
#     features.append(e2_a2_ratio)
    
#     # Ratio of (b2+e2) and a2 [40]
#     b2_e2_a2_ratio = (second_derivative[valleys[0]] + second_derivative[peaks[1]]) / second_derivative[peaks[0]]
#     features.append(b2_e2_a2_ratio)
    
#     # Interval time from the second PPG cycle start point (l2) in second derivative of PPG to a2
#     ta2 = peaks[0] - valleys[0]
#     features.append(ta2)
    
#     # Interval time from point l2 ti b2
#     tb2 = valleys[0] - peaks[0]
#     features.append(tb2)
    
#     # Ratio of ta1 and tpi
#     ta1_tpi_ratio = ta1 / pulse_interval
#     features.append(ta1_tpi_ratio)
    
#     # Ratio of tb1 and tpi
#     tb1_tpi_ratio = tb1 / pulse_interval
#     features.append(tb1_tpi_ratio)
    
#     # Ratio of te1 and tpi
#     te1_tpi_ratio = (valleys[1] - peaks[0]) / sampling_rate / pulse_interval
#     features.append(te1_tpi_ratio)
    
#     # Ratio of time interval of l1 (tl1) and tpi
#     tl1_tpi_ratio = valleys[0] / sampling_rate / pulse_interval
#     features.append(tl1_tpi_ratio)
    
#     # Ratio of ta2 and tpi
#     ta2_tpi_ratio = ta2 / pulse_interval
#     features.append(ta2_tpi_ratio)
    
#     # Ratio of tb2 and tpi
#     tb2_tpi_ratio = tb2 / pulse_interval
#     features.append(tb2_tpi_ratio)
    
#     # Ratio of ta1+ta2 and pulse interval (tpi)
#     ta1_ta2_tpi_ratio = (ta1 + ta2) / pulse_interval
#     features.append(ta1_ta2_tpi_ratio)
    
#     # Ratio of (tb1+tb2) and pulse interval (tpi)
#     tb1_tb2_tpi_ratio = (tb1 + tb2) / pulse_interval 
#     features.append(tb1_tb2_tpi_ratio)
       
#     # Ratio of (te1+t2) and pulse interval (tpi)
#     te1_te2_tpi_ratio = (te1_tpi_ratio + tb2_tpi_ratio)
#     features.append(te1_te2_tpi_ratio)
    
#     # Ratio of tl1+t3 and pulse interval (tpi)
#     tl1_t3_tpi_ratio = (tl1_tpi_ratio + t3_tpi_ratio)
#     features.append(tl1_t3_tpi_ratio)
    
#     # Fundamental component frequency obtained from Fast Fourier Transformation (FFT)
#     fbase = np.argmax(fft(smoothed_signal))
#     features.append(fbase)
    
#     # Fundamental component magnitude from FFT
#     fbase_magnitude = np.max(fft(smoothed_signal))
#     features.append(fbase_magnitude)
    
#     # Second component frequency obtained from FFT. Such that, fbase<f2nd
#     f2nd = np.argmax(fft(smoothed_signal[fbase:]))
#     features.append(f2nd)
    
#     # Second component magnitude from FFT
#     f2nd_magnitude = np.max(fft(smoothed_signal[fbase:]))
#     features.append(f2nd_magnitude)
    
#     # Third component frequency obtained from FFT. Such that, fbase<f2nd<f3rd
#     f3rd = np.argmax(fft(smoothed_signal[f2nd:]))
#     features.append(f3rd)
    
#     # Third component magnitude acquired from FFT
#     f3rd_magnitude = np.max(fft(smoothed_signal[f2nd:]))
#     features.append(f3rd_magnitude)

#         # Append features to the list


#     all_features.append(features)

#     return all_features

# # Example usage
# sampling_rate = 1000  # Sample rate in Hz

# # Assuming merged_signals is a list containing merged signals for each PPG signal
# # merged_signals = [merged_signal1, merged_signal2, merged_signal3, ...]

# # Extract features for each merged signal
# extracted_features = extract_features(merged_signal, sampling_rate)

# # Visualize or process the extracted features as needed
